In [1]:
import warnings
warnings.filterwarnings("ignore")
from pathlib import Path
from tbdynamics.calibration.runner import calibrate, calibrate_with_configs
import pymc as pm
from estival.wrappers import pymc as epm
from estival.sampling import tools as esamp
from estival.wrappers import nevergrad as eng
from estival.utils.parallel import map_parallel
import nevergrad as ng
from tbdynamics.camau.calibration.utils import get_bcm
from estival.utils.sample import SampleTypes

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
covid_configs = {
        'no_covid': {
            "detection_reduction": False,
            "contact_reduction": False
        },  # No reduction
        'detection': {
            "detection_reduction": True,
            "contact_reduction": False
        },  # No contact reduction
        'contact': {
            "detection_reduction": False,
            "contact_reduction": True
        },  # Only contact reduction
        'detection_and_contact': {
            "detection_reduction": True,
            "contact_reduction": True
        },  # With detection + contact reduction
    }

covid_effects = {
    'detection_reduction':True,
    'contact_reduction':False
}
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
    "screening_scaleup_shape": 0.3,
    "screening_inflection_time": 1993,
    # "acf_sensitivity": 0.90,
    # "prop_mixing_same_stratum": 0.5,
}
N_CHAINS = 4

In [3]:
OUT_PATH = Path.cwd().parent.parent / 'data/outputs/camau/r3008'
OUT_PATH.mkdir(parents=True, exist_ok=True)

In [4]:
calibrate(OUT_PATH,params,covid_effects,n_chains=N_CHAINS, draws=150000, tune=50000)

Multiprocess sampling (4 chains in 4 jobs)
DEMetropolisZ: [contact_rate, rr_infection_latent, rr_infection_recovered, early_prop_adjuster, late_reactivation_adjuster, smear_positive_death_rate, smear_negative_death_rate, smear_positive_self_recovery, smear_negative_self_recovery, incidence_props_pulmonary, incidence_props_smear_positive_among_pulmonary, time_to_screening_end_asymp, acf_sensitivity, prop_mixing_same_stratum, detection_reduction, total_population_dispersion, notif_dispersion, latent_dispersion, act3_trial_dispersion, act3_control_dispersion]


Sampling 4 chains for 50_000 tune and 150_000 draw iterations (200_000 + 600_000 draws total) took 47226 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
